# Datasets

In [1]:
import os
from typing import List

import numpy as np
import pywsra
import xarray as xr
from littlebuoybigwaves import geo

from configure import get_config


# Setup

Load the configuration file, `config.toml`, which contains the data directories.

In [2]:
config = get_config()

## WSRA and P-3 met data

In [3]:
WSRA_DIR = config['dir']['wsra']

def construct_wsra_path(storm_name: str) -> str:
    return os.path.join(WSRA_DIR, storm_name)

def read_wsra_and_met_data(
    storm_name: str,
    met_data_vars:  str | List[str] = 'all',
    met_rename_dict: dict[str, str] | None = None,
) -> xr.Dataset:
    # Open WSRA and P-3 met data.
    directory = construct_wsra_path(storm_name)
    wsra_ds = pywsra.read_wsra_directory(directory, index_by_time=True)
    met_ds = pywsra.read_met_directory(os.path.join(directory, 'met'),  # TODO: use AC data?
                                       data_vars=met_data_vars)

    # Merge the datasets by resampling the P-3 met data onto the WSRA times.
    wsra_merged_ds = pywsra.merge_met_vars(wsra_ds=wsra_ds,
                                           met_ds=met_ds,
                                           data_vars=met_data_vars,
                                           resample_method=np.nanmedian,
                                           rename_dict=met_rename_dict)

    # Save `storm_name` as an attribute for future reference.
    wsra_merged_ds.attrs['storm_name'] = storm_name.lower()

    return wsra_merged_ds


In [4]:
read_kwargs = dict(
    met_data_vars = [
        'SfmrWS.1', 'SfmrWErr.1', 'SfmrRainRate.1', 'SfmrDV.1', 'LonGPS.1', 'LatGPS.1',
    ],
    met_rename_dict = {
        'SfmrWS.1': 'met_sfmr_10m_wind_speed',
        'SfmrWErr.1': 'met_sfmr_10m_wind_speed_error',
        'SfmrRainRate.1': 'met_sfmr_rain_rate',
        'SfmrDV.1': 'met_sfmr_data_validity',
        'LonGPS.1': 'met_longitude',
        'LatGPS.1': 'met_latitude',
    },
)

In [9]:
earl_ds = read_wsra_and_met_data('Earl', **read_kwargs)
fiona_ds = read_wsra_and_met_data('Fiona', **read_kwargs)
ian_ds = read_wsra_and_met_data('Ian', **read_kwargs)
# julia_ds = read_wsra_and_met_data('Julia', **read_kwargs)  #TODO: wrong data on Prosensing site for Julia 
idalia_ds = read_wsra_and_met_data('Idalia', **read_kwargs)
lee_ds = read_wsra_and_met_data('Lee', **read_kwargs)

/Users/jacob/Programs/pywsra/src/pywsra/met.py:166: RuntimeWarning: All-NaN slice encountered
  values.append(resample_method(met_in_window[var].values))
/Users/jacob/Programs/pywsra/src/pywsra/met.py:166: RuntimeWarning: All-NaN slice encountered
  values.append(resample_method(met_in_window[var].values))
/Users/jacob/Programs/pywsra/src/pywsra/met.py:166: RuntimeWarning: All-NaN slice encountered
  values.append(resample_method(met_in_window[var].values))
/Users/jacob/Programs/pywsra/src/pywsra/met.py:166: RuntimeWarning: All-NaN slice encountered
  values.append(resample_method(met_in_window[var].values))
/Users/jacob/Programs/pywsra/src/pywsra/met.py:166: RuntimeWarning: All-NaN slice encountered
  values.append(resample_method(met_in_window[var].values))
/opt/homebrew/Caskroom/miniconda/base/envs/wsra-mss/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


## NHC

In [6]:
NHC_DIR = config['dir']['nhc']

def construct_nhc_path(storm_id: str, feature: str) -> str:
    folder = f'{storm_id.lower()}_best_track'
    filename = f'{storm_id.upper()}_{feature}.shp'
    return os.path.join(NHC_DIR, folder, filename)

def read_nhc_best_track(storm_id):
    pts = geo.read_shp_file(construct_nhc_path(storm_id, 'pts'), index_by_datetime=True)
    pts = geo.best_track_pts_to_intensity(pts)
    lin = geo.read_shp_file(construct_nhc_path(storm_id, 'lin'))
    windswath = geo.read_shp_file(construct_nhc_path(storm_id, 'windswath'))
    # radii = geo.read_shp_file(construct_nhc_path(storm_id, 'radii'))
    return pts, lin, windswath #, radii


In [7]:
earl_best_track = read_nhc_best_track(storm_id=earl_ds.attrs['storm_id'])
fiona_best_track = read_nhc_best_track(storm_id=fiona_ds.attrs['storm_id'])
ian_best_track = read_nhc_best_track(storm_id=ian_ds.attrs['storm_id'])
# julia_best_track = read_nhc_best_track(storm_id=julia_ds.attrs['storm_id'])
idalia_best_track = read_nhc_best_track(storm_id=idalia_ds.attrs['storm_id'])
lee_best_track = read_nhc_best_track(storm_id=lee_ds.attrs['storm_id'])

## Store

In [8]:
%%capture

%store earl_ds
%store fiona_ds
%store ian_ds
# %store julia_ds
%store idalia_ds
%store lee_ds


%store earl_best_track
%store fiona_best_track
%store ian_best_track
# %store julia_best_track
%store idalia_best_track
%store lee_best_track